In [1]:
"""Hello Analytics Reporting API V4."""

from apiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import numpy as np

SCOPES = ['https://www.googleapis.com/auth/analytics.readonly']
KEY_FILE_LOCATION = 'AFN-Google-Analytics-aaec7e7fd2f2.json'
VIEW_ID = '201412152'

In [2]:
def initialize_analyticsreporting():
  """Initializes an Analytics Reporting API V4 service object.

  Returns:
    An authorized Analytics Reporting API V4 service object.
  """
  credentials = ServiceAccountCredentials.from_json_keyfile_name(
      KEY_FILE_LOCATION, SCOPES)

  # Build the service object.
  analytics = build('analyticsreporting', 'v4', credentials=credentials)

  return analytics

In [3]:
def get_report(analytics):
  """Queries the Analytics Reporting API V4.

  Args:
    analytics: An authorized Analytics Reporting API V4 service object.
  Returns:
    The Analytics Reporting API V4 response.
  """
  return analytics.reports().batchGet(
      body={
        'reportRequests': [
        {
          'viewId': VIEW_ID,
          'dateRanges': [{'startDate': '7daysAgo', 'endDate': 'today'}],
          'metrics': [{'expression': 'ga:sessions'}, {'expression': 'ga:users'}],
          'dimensions': [{'name': 'ga:country'}, {'name': 'ga:sourceMedium'}]
        }]
      }
  ).execute()

In [4]:
def get_source_medium_report(analytics):
  """Queries the Analytics Reporting API V4.

  Args:
    analytics: An authorized Analytics Reporting API V4 service object.
  Returns:
    The Analytics Reporting API V4 response.
  """
  return analytics.reports().batchGet(
      body={
        'reportRequests': [
        {
          'viewId': VIEW_ID,
          'dateRanges': [{'startDate': '7daysAgo', 'endDate': 'today'}],
          'metrics': [{'expression': 'ga:users'}, {'expression': 'ga:sessions'}],
          'dimensions': [{'name': 'ga:sourceMedium'}]
        }]
      }
  ).execute()

In [5]:
def print_response(response):
  """Parses and prints the Analytics Reporting API V4 response.

  Args:
    response: An Analytics Reporting API V4 response.
  """
  for report in response.get('reports', []):
    columnHeader = report.get('columnHeader', {})
    dimensionHeaders = columnHeader.get('dimensions', [])
    metricHeaders = columnHeader.get('metricHeader', {}).get('metricHeaderEntries', [])

    for row in report.get('data', {}).get('rows', []):
      dimensions = row.get('dimensions', [])
      dateRangeValues = row.get('metrics', [])

      for header, dimension in zip(dimensionHeaders, dimensions):
        print(header + ': ', dimension)

      for i, values in enumerate(dateRangeValues):
        print('Date range:', str(i))
        for metricHeader, value in zip(metricHeaders, values.get('values')):
          print(metricHeader.get('name') + ':', value)

In [6]:
def main():
  analytics = initialize_analyticsreporting()
  response = get_report(analytics)
  print_response(response)

In [7]:
analytics = initialize_analyticsreporting()
response = get_report(analytics)

In [8]:
for report in response.get('reports', []):
    print(report.get('columnHeader'))
    for dimensionName in report.get('columnHeader').get('dimensions'):
        print(dimensionName)
    for metricName in report.get('columnHeader').get('metricHeader').get('metricHeaderEntries'):
        print(metricName.get('type'))

{'dimensions': ['ga:country', 'ga:sourceMedium'], 'metricHeader': {'metricHeaderEntries': [{'name': 'ga:sessions', 'type': 'INTEGER'}, {'name': 'ga:users', 'type': 'INTEGER'}]}}
ga:country
ga:sourceMedium
INTEGER
INTEGER


In [9]:
analytics = initialize_analyticsreporting()
response_country = get_report(analytics)
response_source_medium = get_source_medium_report(analytics)

In [10]:
data_rows = []
col_names = []
metric_types = []

for report in response.get('reports', []):
    columnHeader = report.get('columnHeader', [])
    dimensionHeaders = columnHeader.get('dimensions', [])
    metricHeaders = columnHeader.get('metricHeader', {}).get('metricHeaderEntries', [])

    for dimensionName in dimensionHeaders:
        col_names.append(dimensionName)

    for metricName in metricHeaders:
        col_names.append(metricName.get('name'))

    for metricType in metricHeaders:
        metric_types.append(metricType.get('type'))

    for row in report.get('data', {}).get('rows', []):
        dimensions = row.get('dimensions', [])
        metricHeaders = row.get('metrics', [])
        data_row = []

        for dimension in dimensions:
            data_row.append(dimension)

        for idx, value in enumerate(metricHeaders[0].get('values'), start=0):
            if metric_types[idx] == 'INTEGER':
                data_row.append(int(value))
            elif metric_types[idx] == 'FLOAT':
                data_row.append(float(value))
            else:
                data_row.append(value)
        data_rows.append(data_row)

In [11]:
def create_dataframe(response):
    data_rows = []
    col_names = []

    for report in response.get('reports', []):
        columnHeader = report.get('columnHeader', [])
        dimensionHeaders = columnHeader.get('dimensions', [])
        metricHeaders = columnHeader.get('metricHeader', {}).get('metricHeaderEntries', [])

        for dimensionName in dimensionHeaders:
            col_names.append(dimensionName)

        for metricName in metricHeaders:
            col_names.append(metricName.get('name'))

        for metricType in metricHeaders:
            metric_types.append(metricType.get('type'))

        for row in report.get('data', {}).get('rows', []):
            dimensions = row.get('dimensions', [])
            metricHeaders = row.get('metrics', [])
            data_row = []

            for dimension in dimensions:
                data_row.append(dimension)

            for idx, value in enumerate(metricHeaders[0].get('values'), start=0):
                if metric_types[idx] == 'INTEGER':
                    data_row.append(int(value))
                elif metric_types[idx] == 'FLOAT':
                    data_row.append(float(value))
                else:
                    data_row.append(value)

            data_rows.append(data_row)

    return pd.DataFrame(data_rows, columns=col_names)

In [12]:
df_country = create_dataframe(response_country)

In [13]:
df_source_medium = create_dataframe(response_source_medium)

In [14]:
df_country.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 638 entries, 0 to 637
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ga:country       638 non-null    object
 1   ga:sourceMedium  638 non-null    object
 2   ga:sessions      638 non-null    int64 
 3   ga:users         638 non-null    int64 
dtypes: int64(2), object(2)
memory usage: 20.1+ KB


In [15]:
df_source_medium.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 139 entries, 0 to 138
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ga:sourceMedium  139 non-null    object
 1   ga:users         139 non-null    int64 
 2   ga:sessions      139 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ KB


In [18]:
df_country[df_country['ga:country'] == 'Indonesia'].sort_values('ga:users', ascending=False)

,ga:country,ga:sourceMedium,ga:sessions,ga:users
199,Indonesia,facebook / paid,97968,77988
202,Indonesia,google / organic,15191,13705
185,Indonesia,(direct) / (none),1768,1566
223,Indonesia,tv / qr-code,864,523
197,Indonesia,facebook / (not set),381,337
216,Indonesia,m.facebook.com / referral,128,124
206,Indonesia,instagram / organic,167,122
224,Indonesia,vst.to / referral,167,105
198,Indonesia,facebook / organic,47,43
211,Indonesia,l.facebook.com / referral,20,20


In [17]:
df_source_medium

,ga:sourceMedium,ga:users,ga:sessions
0,(direct) / (none),13860,14969
1,10.225.0.24:4503 / referral,1,2
2,10.225.1.124:4503 / referral,1,13
3,151.ddbstaging.com / referral,1,9
4,adk / facebook,1,1
...,...,...,...
134,workona.com / referral,1,2
135,www-pinterest-com.cdn.ampproject.org / referral,3,3
136,yahoo / organic,159,177
137,youtube.com / referral,1,1
